In [1]:
import tensorflow as tf
#https://medium.com/%E8%BB%9F%E9%AB%94%E4%B9%8B%E5%BF%83/%E9%81%A9%E5%90%88%E5%A4%A7%E9%87%8F%E8%B3%87%E6%96%99i-o%E7%9A%84%E5%84%B2%E5%AD%98%E6%A0%BC%E5%BC%8F-tfrecord%E7%B0%A1%E4%BB%8B%E8%88%87%E6%93%8D%E4%BD%9C%E6%95%99%E5%AD%B8-cd27e50d51ee

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    # because args is alreaddy a list, no need to use []
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [23]:
"""make a synthesized dataset and its offset table"""
import random
import sys
import struct
import os
name = []
gpa = []
rank = []

random.seed(0)

for i in range(100):
    name_now=('haha'+str(i)).encode('utf-8')
    name.append(name_now)
    # random length gpa: list
    gpas=[]
    n_course=random.randint(1,10)    
    for i in range(n_course):
        gpas.append(3.0+i*0.01)
    gpa.append(gpas)
    rank.append(100-i)

def serialize_example(_name, _gpa, _rank):
    feature = {
        "name": _bytes_feature(_name),
        "gpa": _float_feature(_gpa),
        "rank": _int64_feature(_rank),
        }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    serialized_example=example_proto.SerializeToString() 
    #length=sys.getsizeof(serialized_example)
    #return length, serialized_example
    return serialized_example
# build offset_table while writing TFRecords, int64 for offset of each instance, e.g. offset of instance #100 is 100*8B=800B

# byte is little endian
offset_table=open("1d-sparse-example_offset_table",'wb') 
with tf.io.TFRecordWriter("1d-sparse-example.tfrecords") as writer:
    for _name, _gpa, _rank in zip(name, gpa, rank):
        # this getfilesize method is so useful, filesize is offset
        current_filesize=os.path.getsize("1d-sparse-example.tfrecords")
        print("offset=",current_filesize)
        offset_table.write(current_filesize.to_bytes(8, byteorder = 'little'))
        #length,serialized_example = serialize_example(_name, _gpa, _rank)
        serialized_example = serialize_example(_name, _gpa, _rank)        
        writer.write(serialized_example)
        
        
offset_table.close()

offset= 0
offset= 93
offset= 186
offset= 255
offset= 340
offset= 441
offset= 538
offset= 631
offset= 716
offset= 813
offset= 902
offset= 1008
offset= 1090
offset= 1192
offset= 1270
offset= 1356
offset= 1434
offset= 1508
offset= 1614
offset= 1700
offset= 1802
offset= 1908
offset= 1986
offset= 2072
offset= 2146
offset= 2220
offset= 2310
offset= 2408
offset= 2510
offset= 2584
offset= 2674
offset= 2768
offset= 2858
offset= 2964
offset= 3046
offset= 3148
offset= 3246
offset= 3344
offset= 3446
offset= 3532
offset= 3602
offset= 3704
offset= 3774
offset= 3848
offset= 3942
offset= 4012
offset= 4118
offset= 4216
offset= 4306
offset= 4388
offset= 4478
offset= 4552
offset= 4634
offset= 4740
offset= 4822
offset= 4904
offset= 4982
offset= 5084
offset= 5182
offset= 5256
offset= 5330
offset= 5420
offset= 5522
offset= 5620
offset= 5694
offset= 5780
offset= 5882
offset= 5968
offset= 6042
offset= 6144
offset= 6234
offset= 6336
offset= 6418
offset= 6524
offset= 6626
offset= 6732
offset= 6818
offset= 6916


In [24]:
"""how to parse the tfrecord file"""

dataset = tf.data.TFRecordDataset('1d-sparse-example.tfrecords')

image_feature_description = {
    'name': tf.io.FixedLenFeature([], tf.string),
    # var len is different
    'gpa': tf.io.VarLenFeature(tf.float32),
    'rank': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_function(example_proto):
    serialized_example = tf.io.parse_single_example(example_proto, image_feature_description)
    return serialized_example

for idx, features in enumerate(dataset):
    print(f"reading student {idx}...")
    parsed_features = _parse_function(features)
    for key in parsed_features:
        # var len needs different way to show, not just numpy()
        if key=='gpa':
            print(parsed_features[key].values)
        else:
            print(f"{key}: {parsed_features[key].numpy()}")

reading student 0...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06], shape=(7,), dtype=float32)
name: b'haha0'
rank: 94
reading student 1...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06], shape=(7,), dtype=float32)
name: b'haha1'
rank: 94
reading student 2...
tf.Tensor([3.], shape=(1,), dtype=float32)
name: b'haha2'
rank: 100
reading student 3...
tf.Tensor([3.   3.01 3.02 3.03 3.04], shape=(5,), dtype=float32)
name: b'haha3'
rank: 96
reading student 4...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06 3.07 3.08], shape=(9,), dtype=float32)
name: b'haha4'
rank: 92
reading student 5...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06 3.07], shape=(8,), dtype=float32)
name: b'haha5'
rank: 93
reading student 6...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06], shape=(7,), dtype=float32)
name: b'haha6'
rank: 94
reading student 7...
tf.Tensor([3.   3.01 3.02 3.03 3.04], shape=(5,), dtype=float32)
name: b'haha7'
rank: 96
reading student 8...
tf.Tensor([3.   3.01 3.02 3.03 3.04 3.05 3.06 3.07], sh

In [4]:
"""offset table read test"""

oft=open("1d-sparse-example_offset_table",'rb')
idx=5
for i in range (100):
    tmp=oft.read(8)
    #print(tmp)
    length=struct.unpack("<Q",tmp)
    print(length[0])

0
110
220
306
408
526
640
750
852
966
1072
1195
1294
1413
1508
1611
1706
1797
1920
2023
2142
2265
2360
2463
2554
2645
2752
2867
2986
3077
3184
3295
3402
3525
3624
3743
3858
3973
4092
4195
4282
4401
4488
4579
4690
4777
4900
5015
5122
5221
5328
5419
5518
5641
5740
5839
5934
6053
6168
6259
6350
6457
6576
6691
6782
6885
7004
7107
7198
7317
7424
7543
7642
7765
7884
8007
8110
8225
8316
8439
8550
8657
8780
8879
8982
9077
9176
9271
9358
9481
9584
9699
9790
9881
9976
10071
10158
10249
10368
10479


In [26]:
"""random reader function"""

def sparse_random_read(binfile, oft, idx):
    # read offset, 8B per instance
    #TODO: oft should be loaded to mem
    oft.seek(idx*8,0)
    tmp=oft.read(8)
    offset=struct.unpack("<Q",tmp)[0]
    print("offset=",offset)
    
    # read data length
    binfile.seek(offset,0)
    tmp=binfile.read(8)
    length=struct.unpack("<Q",tmp)[0]
    print("length=",length)
    # we already read first 8B
    record_l_from_col2=length+8
    
    # random read
    tmp=binfile.read(record_l_from_col2)
    r_data=tmp[4:-4]

    # deserialize data
    parsed_features = _parse_function(r_data)
    return parsed_features
    
    

In [30]:
import struct
binfile=open('1d-sparse-example.tfrecords','rb')
oftfile=open('1d-sparse-example_offset_table','rb')
a=sparse_random_read(binfile,oftfile, 0)
print(a)

offset= 0
length= 77
93
{'gpa': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x000001D9A3CA38B0>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'haha0'>, 'rank': <tf.Tensor: shape=(), dtype=int64, numpy=94>}


In [ ]:
"""basic test: read instance#0"""

import struct
#https://www.kite.com/python/answers/how-to-read-bytes-from-a-binary-file-in-python
#https://docs.python.org/3/library/struct.html

#uint64 length (little endian)
#uint32 masked_crc32_of_length
#byte   data[length]
#uint32 masked_crc32_of_data
binfile=open('1d-sparse-example.tfrecords','rb')
binfile.seek(0)

# read data length
tmp=binfile.read(8)
length=struct.unpack("<Q",tmp)
print("length=",length[0])

# skip crc
binfile.seek(4,1)

# read data 
tmp=binfile.read(length[0])
parsed_features = _parse_function(tmp)
print("parsed_data=",parsed_features)
print("parsed_data=",parsed_features['gpa'])
# skip crc
binfile.seek(4,1)
binfile.close()